In [1]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Install required packages
%pip install tensorflow==2.9.0
%pip install matplotlib mtcnn opencv-python pillow

ERROR: Could not find a version that satisfies the requirement tensorflow==2.9.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.9.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
# Step 1: Import all necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Step 2: Import your BMI predictor
from bmi_prediction import BMIPredictor

# Step 3: Define your paths (same as in your main.py)
DATA_PATH = "/Users/casey/Documents/GitHub/Face_to_BMI/data.csv"
IMAGE_PATH = "/Users/casey/Documents/GitHub/Face_to_BMI/Images"
MODEL_PATH = "/Users/casey/Documents/GitHub/Face_to_BMI/saved_model.h5"  # This is the saved model file

# Step 4: Check if model exists
if not os.path.exists(MODEL_PATH):
    print(f"Error: Model not found at {MODEL_PATH}. Train the model first.")
else:
    print(f"Model found at {MODEL_PATH}")

Error: Model not found at /Users/casey/Documents/GitHub/Face_to_BMI/saved_model.h5. Train the model first.


In [14]:
# Step 5: Initialize the predictor
predictor = BMIPredictor(MODEL_PATH)

# Step 6: Load the dataset
df = pd.read_csv(DATA_PATH)
print(f"Dataset loaded with {len(df)} entries")

# Step 7: Create a test subset (20% of the data)
test_df = df.sample(frac=0.2, random_state=42)
print(f"Selected {len(test_df)} images for testing")

# Step 8: Create a dictionary to store results
results = []

Loading model from /Users/casey/Documents/GitHub/Face_to_BMI/


ValueError: File format not supported: filepath=/Users/casey/Documents/GitHub/Face_to_BMI/. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/Users/casey/Documents/GitHub/Face_to_BMI/, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [4]:
# Step 9: Loop through test images and predict BMI
for idx, row in test_df.iterrows():
    image_path = os.path.join(IMAGE_PATH, row['name'])
    
    if not os.path.exists(image_path):
        print(f"Warning: Image {image_path} not found")
        continue
    
    # Get actual BMI
    actual_bmi = row['bmi']
    
    # Predict BMI
    predicted_bmi, category = predictor.predict_bmi(image_path=image_path)
    
    if predicted_bmi is not None:
        # Calculate error
        error = abs(predicted_bmi - actual_bmi)
        
        results.append({
            'image': row['name'],
            'actual_bmi': actual_bmi,
            'predicted_bmi': predicted_bmi,
            'category': category,
            'error': error
        })
        
        print(f"Image: {row['name']}")
        print(f"  Actual BMI: {actual_bmi:.2f}")
        print(f"  Predicted BMI: {predicted_bmi:.2f}")
        print(f"  Error: {error:.2f}")
        print(f"  Category: {category}")
        print("---")

# Step 10: Calculate overall metrics
if results:
    errors = [r['error'] for r in results]
    mean_error = sum(errors) / len(errors)
    max_error = max(errors)
    min_error = min(errors)
    
    print("\nTest Results Summary:")
    print(f"Number of test images: {len(results)}")
    print(f"Mean Absolute Error: {mean_error:.2f}")
    print(f"Max Error: {max_error:.2f}")
    print(f"Min Error: {min_error:.2f}")
    
    # Create a DataFrame with results
    results_df = pd.DataFrame(results)
    
    # Save results to CSV
    results_df.to_csv("test_results.csv", index=False)
    print("Detailed results saved to test_results.csv")

# Step 11: Visualize the results
if results:
    plt.figure(figsize=(15, 10))
    
    # 1. Plot predicted vs actual BMI
    plt.subplot(2, 2, 1)
    plt.scatter([r['actual_bmi'] for r in results], [r['predicted_bmi'] for r in results])
    plt.plot([min([r['actual_bmi'] for r in results]), max([r['actual_bmi'] for r in results])], 
             [min([r['actual_bmi'] for r in results]), max([r['actual_bmi'] for r in results])], 'r--')
    plt.xlabel('Actual BMI')
    plt.ylabel('Predicted BMI')
    plt.title('Predicted vs Actual BMI')
    
    # 2. Plot error distribution
    plt.subplot(2, 2, 2)
    plt.hist(errors, bins=10)
    plt.xlabel('Absolute Error')
    plt.ylabel('Count')
    plt.title('Error Distribution')
    
    # 3. Plot error vs actual BMI
    plt.subplot(2, 2, 3)
    plt.scatter([r['actual_bmi'] for r in results], [r['error'] for r in results])
    plt.xlabel('Actual BMI')
    plt.ylabel('Absolute Error')
    plt.title('Error vs Actual BMI')
    
    # 4. Show some sample image predictions
    if len(results) > 0:
        # Get images with lowest and highest errors
        sorted_results = sorted(results, key=lambda x: x['error'])
        best_prediction = sorted_results[0]
        worst_prediction = sorted_results[-1]
        
        plt.subplot(2, 2, 4)
        plt.axis('off')
        plt.title('Sample Results')
        info_text = f"Best prediction:\n"
        info_text += f"Image: {best_prediction['image']}\n"
        info_text += f"Actual: {best_prediction['actual_bmi']:.2f}, Predicted: {best_prediction['predicted_bmi']:.2f}\n"
        info_text += f"Error: {best_prediction['error']:.2f}\n\n"
        
        info_text += f"Worst prediction:\n"
        info_text += f"Image: {worst_prediction['image']}\n"
        info_text += f"Actual: {worst_prediction['actual_bmi']:.2f}, Predicted: {worst_prediction['predicted_bmi']:.2f}\n"
        info_text += f"Error: {worst_prediction['error']:.2f}"
        
        plt.text(0.1, 0.5, info_text, fontsize=10)
    
    plt.tight_layout()
    plt.savefig("test_results_visualization.png")
    plt.show()

NameError: name 'test_df' is not defined